<a href="https://colab.research.google.com/github/SandaAbhishekSagar/Transformer/blob/main/Transformer_Wyckoff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

df = pd.read_csv('Wyckoff_QA_Complete_1000.csv')
print(df.head())
print('Columns:', df.columns.tolist())
print('Number of rows:', len(df))


                                Question  \
0     What is a Spring in Wyckoff terms?   
1     What is a Spring in Wyckoff terms?   
2              What is an Upthrust (UT)?   
3            What is the Wyckoff Method?   
4  What is the Law of Effort vs. Result?   

                                              Answer  
0  A Spring is a false breakout to the downside d...  
1  A Spring is a false breakout to the downside d...  
2  An Upthrust is a false breakout to the upside ...  
3  The Wyckoff Method is a technical analysis app...  
4  This law states that price movement should be ...  
Columns: ['Question', 'Answer']
Number of rows: 1015


In [4]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Load the dataset
df = pd.read_csv('Wyckoff_QA_Complete_1000.csv')

# Format input and target texts for T5
df['input_text'] = 'question: ' + df['Question'].astype(str)
df['target_text'] = df['Answer'].astype(str)

# Split into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df[['input_text', 'target_text']])
val_dataset = Dataset.from_pandas(val_df[['input_text', 'target_text']])


In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Initialize model and tokenizer
model_name = 't5-small'  # You can use 't5-base' for better performance
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Define tokenization function
def tokenize_function(examples):
    # Tokenize inputs
    model_inputs = tokenizer(examples['input_text'], max_length=512,
                             truncation=True, padding='max_length')

    # Tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['target_text'], max_length=512,
                           truncation=True, padding='max_length')

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/102 [00:00<?, ? examples/s]

In [8]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq
import torch

# Output directory for model
output_dir = "./wyckoff_chatbot_model"

# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    fp16=torch.cuda.is_available(),
    logging_dir='./logs',
    logging_steps=10
)

# Data collator for sequence-to-sequence tasks
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding=True
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-8-4164f37b410a>:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
# Train the model
trainer.train()

# Save the trained model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sabhisheksagar200 (sabhisheksagar200-northeastern-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.370200,0.291637
2,0.250100,0.212011
3,0.211400,0.165195
4,0.179600,0.144194
5,0.185000,0.138078


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


('./wyckoff_chatbot_model/tokenizer_config.json',
 './wyckoff_chatbot_model/special_tokens_map.json',
 './wyckoff_chatbot_model/spiece.model',
 './wyckoff_chatbot_model/added_tokens.json')

In [10]:
def wyckoff_chatbot(question, model_dir="./wyckoff_chatbot_model"):
    """Generate an answer to a question about Wyckoff methodology"""
    # Load the saved model and tokenizer
    tokenizer = T5Tokenizer.from_pretrained(model_dir)
    model = T5ForConditionalGeneration.from_pretrained(model_dir)

    # Format input text
    input_text = f"question: {question}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate answer
    output = model.generate(
        input_ids,
        max_length=256,
        num_beams=5,
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    # Decode and return
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer

# Interactive interface
def interactive_chatbot():
    print("\n=== Wyckoff Trading Methodology Chatbot ===")
    print("Ask questions about Wyckoff trading (type 'exit' to quit)")

    while True:
        user_question = input("\nYour question: ")
        if user_question.lower() in ['exit', 'quit', 'bye']:
            print("Thank you for using the Wyckoff chatbot!")
            break

        answer = wyckoff_chatbot(user_question)
        print(f"\nWyckoff Chatbot: {answer}")


In [11]:
# Test with sample questions
test_questions = [
    "What is a Spring in Wyckoff terms?",
    "What is the Wyckoff Method?",
    "How do you identify accumulation?",
    "What are the Wyckoff laws?"
]

for question in test_questions:
    answer = wyckoff_chatbot(question)
    print(f"\nQ: {question}")
    print(f"A: {answer}")



Q: What is a Spring in Wyckoff terms?
A: Spring is a Spring in Wyckoff terms.

Q: What is the Wyckoff Method?
A: Wyckoff Method is a tool that helps traders understand market behavior and anticipate potential failures.

Q: How do you identify accumulation?
A: Accumulation is a process that helps you identify accumulation by identifying accumulation.

Q: What are the Wyckoff laws?
A: Wyckoff laws are governed by a set of rules and regulations that govern trade and trade.
